In [ ]:
import os
import sys
import glob
import json
import pytest
import numpy as np
from py.path import local
import warnings

sys.path.append('../..')
from seismiqb import SeismicGeometry

warnings.filterwarnings("ignore")

In [ ]:
# Constants and parameters:
ATTRIBUTES_NAMES = ['snr', 'std_matrix', 'quality_map', 'quality_grid']
CUBE_PATH = os.path.join(SAVING_DIR, 'tmp/', CUBE_NAME)

In [ ]:
# Data for tests:
with open(os.path.join(SAVING_DIR, f'tmp/test_array_{DATESTAMP}.npy'), 'rb') as infile:
    data_array = np.load(infile)

timings_file = glob.glob(os.path.join(TEST_DIR, 'timings*.json'))[0]

with open(timings_file, "r") as infile:
    standard_timings = json.load(infile)

# Tests

In [ ]:
%%time
geometry = SeismicGeometry(CUBE_PATH)

In [ ]:
expected_timings = standard_timings[geometry.format]

In [ ]:
# Check data
geometry_data = geometry[:, :, :].ravel()

if not geometry.quantized:
    assert (geometry_data == data_array.ravel()).all()
else:
    corr = np.corrcoef(geometry_data, data_array.ravel())[0, 1]
    assert corr >=0.9

In [ ]:
# Attributes loading
geometry.make_quality_map(
    [0.1], ['support_js', 'support_hellinger'], safe_strip=0
) # safe_strip=0 because data is a noise and otherwise haven't good traces

for attr_name in ATTRIBUTES_NAMES:
    attr = geometry.__getattr__(key=attr_name)
    assert attr is not None

    attr = geometry.load_attribute(src=attr_name)
    assert attr is not None

In [ ]:
# Slide loading
axis = np.random.randint(3)
loc = np.random.randint(geometry.cube_shape[axis])
data_slice = [slice(None) for i in range(axis)]
data_slice.append(loc)

geometry_slide = geometry.load_slide(loc=loc, axis=axis)
data_slide = data_array[tuple(data_slice)]

if not geometry.quantized:
    assert (geometry_slide == data_slide).all()
else:
    corr = np.corrcoef(geometry_slide.ravel(), data_slide.ravel())[0, 1]
    assert corr >=0.9

In [ ]:
# Crop loading
point = np.random.randint(geometry.cube_shape) // 2
shape = np.random.randint((5, 5, 5), (200, 200, 200))
locations = [slice(start_, np.clip(start_+shape_, 0, max_shape))
             for start_, shape_, max_shape in zip(point, shape, geometry.cube_shape)]

geometry_crop = geometry.load_crop(locations=locations)
data_crop = data_array[tuple(locations)]

if not geometry.quantized:
    assert (geometry_crop == data_crop).all()
else:
    corr = np.corrcoef(geometry_crop.ravel(), data_crop.ravel())[0, 1]
    assert corr >=0.9

In [ ]:
# Visualizations
axis = np.random.randint(3)
loc = np.random.randint(geometry.cube_shape[axis])

geometry.show_slide(loc=loc, axis=axis, figsize=FIGSIZE)  
geometry.show_histogram(hist_log=True, figsize=FIGSIZE)

geometry.show(matrix='mean_matrix', figsize=FIGSIZE)
geometry.show(matrix='snr', figsize=FIGSIZE)

geometry.show_quality_map(figsize=FIGSIZE)    
geometry.show_quality_grid(figsize=FIGSIZE)        

In [ ]:
# Timings
timings_ = geometry.benchmark(n_slide=N_SLIDE, n_crop=N_CROP, use_cache=False, seed=SEED)
timings = {geometry.format: timings_}

with open(os.path.join(SAVING_DIR, f'tmp/timings_{geometry.format}_{DATESTAMP}.json'), "w") as outfile:
    json.dump(timings, outfile)

In [ ]:
# Everything is OK, we can delete the cube:
if DROP_EXTRA_FILES:
    os.remove(CUBE_PATH)